# Q4: Team-Level Stats and Their Relationship to Wins

This notebook analyzes team-level performance metrics and evaluates their contribution to overall success.

We examine metrics such as:

- Team rebounds per game
- Team assists per game
- Team turnovers
- Shooting percentages (FG%, 3P%, FT%)